In [15]:
import os
from dotenv import load_dotenv

load_dotenv() #load api 

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY:
    print("api key found")
else:
    print("api key not found")

api key found


In [16]:

openai_model="gpt-4o-mini"


In [17]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(temperature=0.0,model=openai_model,api_key=OPENAI_API_KEY) # temperature for creativeness 

creative_llm = ChatOpenAI(temperature=0.9, model=openai_model,api_key=OPENAI_API_KEY)

In [18]:
from langchain_core.chat_history import BaseChatMessageHistory # abstract base class that defines the interface for chat message history implementations.
from langchain_core.chat_history import InMemoryChatMessageHistory # concrete implementation of BaseChatMessageHistory that stores messages in memory (RAM)
from langchain_core.runnables.history import RunnableWithMessageHistory # A wrapper that allows you to attach message history to a LangChain Runnable.
from langchain_core.messages import AIMessage # message class representing a message generated by the AI/LLM (Language Model) in a conversation.
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [19]:
result=llm.invoke([
    HumanMessage(content="Hi! I am cristanio"),
    AIMessage(content="Hello cristanio, How can i assist you today"),
    HumanMessage(content="what is my name")]
)

In [20]:
parser=StrOutputParser()
parser.invoke(result)

'Your name is Cristanio. How can I assist you today?'

In [21]:
result.content

'Your name is Cristanio. How can I assist you today?'

In [29]:
store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=InMemoryChatMessageHistory()
    return store[session_id]

In [27]:
config={"configurable": {"session_id":"first_chat"}}

In [30]:
model=RunnableWithMessageHistory(llm, get_session_history)
model.invoke([HumanMessage(content="Hi I am david billa")],config=config).content

'Hello, David Billa! How can I assist you today?'

In [32]:
model.invoke([HumanMessage(content="what is my name")],config=config).content

'Your name is David Billa. How can I help you today?'

In [33]:
store

{'first_chat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am david billa', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, David Billa! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CArnsz0vtkis9kbzVvwxhQOk22CEU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--010c82c9-f51e-4609-9515-bfa643b70d2b-0', usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessag

In [34]:
configg={"configurable":{"session_id":"second_chat"}}

In [36]:
model.invoke([HumanMessage(content="what is my name")],config=configg).content

"I'm sorry, but I don't have access to personal information about you unless you share it with me. How can I assist you today?"

In [38]:
store

{'first_chat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am david billa', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, David Billa! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CArnsz0vtkis9kbzVvwxhQOk22CEU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--010c82c9-f51e-4609-9515-bfa643b70d2b-0', usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessag

In [39]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate

prompt=ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistant. answer all the question to the best of your knowledge"),
    MessagesPlaceholder(variable_name="messages"),
])

In [41]:
chain =prompt | llm

chain.invoke({"messages":["hi i am bob"]}).content

'Hello Bob! How can I assist you today?'

In [43]:
chain.invoke({"messages":[HumanMessage(content="what is my name")]}).content

"I'm sorry, but I don't have access to personal information about you, including your name. How can I assist you today?"

In [44]:
configgg={"configurable":{"session_id":"third_chat"}}

In [45]:
model=RunnableWithMessageHistory(chain, get_session_history)

respone=model.invoke([HumanMessage(content="Hi my name is jim")],config=configgg)

respone.content

'Hello, Jim! How can I assist you today?'

In [46]:
respone=model.invoke([HumanMessage(content="What is my name")],config=configgg)


In [47]:
respone.content

'Your name is Jim. How can I help you today, Jim?'